نصب پکیچ‌های مورد نیاز برای پردازش متن و دانلود فایل‌ دیتاست از گوگل

In [1]:
!pip install -q tensorflow-text==2.6.0
!pip install gdown

     |████████████████████████████████| 4.4 MB 31.3 MB/s 
     |████████████████████████████████| 570.3 MB 318 bytes/s 
     |████████████████████████████████| 132 kB 60.5 MB/s 
     |████████████████████████████████| 5.6 MB 63.8 MB/s 
     |████████████████████████████████| 14.8 MB 62.9 MB/s 
     |████████████████████████████████| 1.3 MB 59.2 MB/s 
     |████████████████████████████████| 462 kB 78.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
cmdstanpy 1.0.4 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.preprocessing import LabelBinarizer

In [3]:
trainfile="train.csv"
testfile="test.csv"
validfile="valid.csv"

دانلود فایل‌ها از منبع گوگل

In [4]:
!gdown --id 1OlcvGWReJMuyYQuOZm149vHWwPtlboR6 --output train.csv
!gdown --id 1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w --output valid.csv
!gdown --id 1ep9H6-HvhB4utJRLVcLzieWNUSG3P_uF --output test.csv 

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1OlcvGWReJMuyYQuOZm149vHWwPtlboR6
To: /content/train.csv
100% 799k/799k [00:00<00:00, 142MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w
To: /content/valid.csv
100% 43.3k/43.3k [00:00<00:00, 60.4MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://dr

بارگذاری فایل‌های دیتاست

In [5]:
traindf = pd.read_csv(trainfile)
validdf = pd.read_csv(validfile)
testdf = pd.read_csv(testfile)

نمونه داده‌های موجود در دیتاست

In [6]:
traindf.head(10)

,text,intent
0,listen to westbam alumb allergic on google music,PlayMusic
1,add step to me to the 50 clásicos playlist,AddToPlaylist
2,i give this current textbook a rating value of...,RateBook
3,play the song little robin redbreast,PlayMusic
4,please add iris dement to my playlist this is ...,AddToPlaylist
5,add slimm cutta calhoun to my this is prince p...,AddToPlaylist
6,i want to listen to seventies music,PlayMusic
7,play a popular chant by brian epstein,PlayMusic
8,find fish story,SearchScreeningEvent
9,book a spot for 3 in mt,BookRestaurant


جداسازی داده ها و تگ‌ها

In [7]:
trainfeatures=traindf.copy()
trainlabels=trainfeatures.pop("intent")
trainfeatures=trainfeatures.values

testfeatures=testdf.copy()
testlabels=testfeatures.pop("intent")
testfeatures=testfeatures.values

validfeatures=validdf.copy()
validlabels=validfeatures.pop("intent")
validfeatures=validfeatures.values


تبدیل متن‌ها و لیبل‌ها به بردار های باینری 


In [8]:
binarizer=LabelBinarizer()
trainlabels=binarizer.fit_transform(trainlabels.values)
testlabels=binarizer.transform(testlabels.values)
validlabels=binarizer.transform(validlabels.values)

دریافت مدل‌های پیش پردازش برت از مخزن تنسورفلو. در این مخزن مدل‌های مختلف برت مانند ALBERT ,
Bert و 
smallBert
قرار دارد که بنا به نیاز می‌توان از هر کدام از آن‌ها استفاده کرد. در این 
پروژه از مدل 
smallBert برای encode 
و از مدل Bert برای پیش پردازش استفاده شده است

In [9]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1'
tfhub_handle_preprocess ='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

text_test = trainfeatures[0]
text_preprocessed = bert_preprocess_model(text_test)

ساختن مدل های بر اساس مدل encoder , pre process

In [10]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)
bert_results = bert_model(text_preprocessed)

کلاس های موجود در دیتاست

In [11]:
binarizer.classes_

array(['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic',
       'RateBook', 'SearchCreativeWork', 'SearchScreeningEvent'],
      dtype='<U20')

ساخت لایه‌های شبکه عصبی با استفاده از مدل‌های پیش پردازش و encoder 
این شبکه عصبی از ۷ لایه dense و یک لایه ورودی برای دسته بندی بردار های ورودی تشکیل شده است

In [13]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(7, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

آموزش مدل ساخته شده

In [14]:
def train():
  classifier_model = build_classifier_model()
  bert_raw_result = classifier_model(tf.constant(trainfeatures[0]))
  classifier_model.summary()


  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  metrics = tf.metrics.CategoricalAccuracy()

  epochs=5
  optimizer=tf.keras.optimizers.Adam(1e-5)
  classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
  print(f'Training model with {tfhub_handle_encoder}')
  history = classifier_model.fit(x=trainfeatures,y=trainlabels,
                               validation_data=(validfeatures,validlabels),
                               batch_size=32,
                               epochs=epochs)
  return classifier_model

تابع تست مدل با استفاده از داده‌های تست و ارزیابی دقت مدل

In [15]:
def test(model):
  loss, accuracy = model.evaluate(testfeatures,testlabels)

  print(f'Loss: {loss}')
  print(f'Accuracy: {accuracy}')

In [ ]:
model = train()

In [ ]:
test(model)

In [ ]:
def print_my_examples(inputs, results):
  result_for_printing = [f'input: {inputs[i]:<30} : estimated intent: {results[i]}' for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()




تست مدل با استفاده از داده‌های ورودی 

In [ ]:
examples = [
    'I like to sing a song at the moment',
    'can you play this amazing song by guitar ?',
    'The weather looks rainy',
    'i give this book a rating value of 1 and a best rating of 6',
    'I want to book a double table for this night'
]

results = tf.nn.softmax(model(tf.constant(examples)))
intents=binarizer.inverse_transform(results.numpy())
print_my_examples(examples, intents)